In [14]:
from datasets import load_dataset
import numpy as np

# ─────────────── Normal full download ───────────────
# ds = load_dataset("rotten_tomatoes")                     # full dataset
# # or
# ds = load_dataset("rotten_tomatoes", split="train")       # only one split

# print(ds)
# for i in range(10):
#     print(ds[i])           # first example
# print(ds.column_names)

a = np.random.randint(0, 100)
print(a)

12


In [18]:
# Python + requests (recommended)
import requests
from pprint import pprint

params = {
    # "dataset": "cornell-movie-review-data/rotten_tomatoes",
    "dataset": "mlfoundations/dclm-baseline-1.0-parquet",
    "config":  "default",
    "split":   "train",
    "offset":  150,
    "length":  10
}

url = "https://datasets-server.huggingface.co/rows"
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print(data.keys())              
    print(len(data["rows"]))        
    print(data["rows"][0])       
else:
    print(response.status_code, response.text)

# url = "https://datasets-server.huggingface.co/size"
# params = {
#     "dataset": "mlfoundations/dclm-baseline-1.0-parquet",
# }
# response = requests.get(url, params=params)

# if response.status_code == 200:
#     data = response.json()
#     pprint(data)
#     splits = data.get("size").get("dataset").get("num_rows")
#     pprint(splits)
    # size = sum(split_info.get("num_rows", 0) for split_info in splits.values())
    # print(size )

dict_keys(['features', 'rows', 'num_rows_total', 'num_rows_per_page', 'partial'])
10
{'row_idx': 150, 'row': {'text': 'This Weekend Steve Martin Also Apologized For A Racist Tweet\n\n\nThis weekend, the world was out to get Justine Sacco fired for her ignorant tweet and failed to notice what Steve Martin was up to.\n\nMartin was having some fun on his Twitter page, discussing the merits of grammar.\xa0 Fans of his would ask him questions about wording and he\'d fire back a funny comment until one particular comment struck people the wrong way.\xa0 @BethanyWedel wrote: "Is this how you spell lasonia?"\xa0 And Martin wrote back...\n\n\n\n\n\nMoments later, unlike Sacco, Steve Martin had the wherewithal to delete the comment but it was too late.\xa0 People started calling Martin out for his remark, which honestly I don\'t even understand.\xa0 Is he making a joke about how an African American would say lasagna?\xa0 Or is he commenting about how they wouldn\'t know how to spell it if it wer

In [3]:
import requests
import json
name = "mlfoundations/dclm-baseline-1.0-parquet"
BASE = "https://datasets-server.huggingface.co/rows"

# ─── 2.A Get first rows (preview) ────────────────────────────────
def get_first_rows(dataset_name, config_name="default", split="train", rows=100):
    url = f"{BASE}"
    params = {
        "dataset": dataset_name,
        "config": config_name,
        "split": split,
        "rows": rows,
    }
    r = requests.get(url, params=params)
    if r.status_code != 200:
        print("Error:", r.status_code, r.text)
        return None
    return r.json()

# Example
preview = get_first_rows("rotten_tomatoes")
if preview:
    print(json.dumps(preview["rows"][:2], indent=2))   # first 2 examples

# ─── 2.B Get parquet URLs (good for large datasets / custom loading) ───
def get_parquet_urls(dataset_name):
    url = f"{BASE}/parquet"
    params = {"dataset": dataset_name}
    # r = requests.get(url, params=params)
    r = requests.get("https://datasets-server.huggingface.co/rows?dataset=cornell-movie-review-data/rotten_tomatoes&config=default&split=train&offset=150&length=10")
    # r = requests.get(f"https://datasets-server.huggingface.co/rows/?dataset=rotten_tomatoes&offset=0&length=100")
    if r.status_code == 200:
        return r.json()
    else:
        print("Error:", r.text)
        return None

# Example
parquet_info = get_parquet_urls(name)
# if parquet_info:
#     print("Parquet files:")
#     for file in parquet_info.get("parquet", []):
#         print(f"• {file['split']} → {file['url']} ({file['size_bytes']/1e6:.1f} MB)")

Error: 404 {"error":"The dataset has been renamed. Please use the current dataset name."}


# Methods to Fetch Data from HuggingFace Datasets-Server

## Available Methods:

1. **`/rows`** - Fetch specific rows by offset and length
2. **`/first-rows`** - Get preview rows (first N rows)
3. **`/parquet`** - Get parquet file URLs (download entire files)
4. **`/size`** - Get dataset size information

In [11]:
# ============================================================================
# Method 1: Fetch Rows by Offset and Length (Most Flexible)
# ============================================================================

def fetch_rows(dataset_name, offset=0, length=100, config=None, split=None):
    """
    Fetch specific rows from HuggingFace datasets-server.
    
    Args:
        dataset_name: Name of the dataset
        offset: Starting row index (default: 0)
        length: Number of rows to fetch (default: 100)
        config: Optional config name
        split: Optional split name (e.g., "train", "test")
    
    Returns:
        JSON response with rows data
    """
    url = f"{BASE}/rows"
    params = {
        "dataset": dataset_name,
        "offset": offset,
        "length": length,
    }
    
    if config:
        params["config"] = config
    if split:
        params["split"] = split
    
    r = requests.get(url, params=params)
    if r.status_code == 200:
        return r.json()
    else:
        print(f"Error {r.status_code}: {r.text}")
        return None

# Example: Fetch first 10 rows
print("=" * 70)
print("Method 1: Fetch Rows by Offset/Length")
print("=" * 70)

dataset_name = "rotten_tomatoes"
data = fetch_rows(dataset_name, offset=0, length=10, split="train")

if data:
    print(f"Dataset: {dataset_name}")
    print(f"Total rows in response: {len(data.get('rows', []))}")
    print(f"\nFirst row:")
    if data.get('rows'):
        first_row = data['rows'][0]
        print(f"  Row data: {first_row.get('row', {})}")
        print(f"  Row index: {first_row.get('row_idx', 'N/A')}")

Method 1: Fetch Rows by Offset/Length
Error 422: {"error":"Parameter 'config' is required"}


In [ ]:
# ============================================================================
# Method 2: Fetch Data in Batches (For Large Datasets)
# ============================================================================

def fetch_dataset_batch(dataset_name, start_idx=0, batch_size=100, split="train"):
    """
    Fetch a batch of data from the dataset.
    
    Args:
        dataset_name: Name of the dataset
        start_idx: Starting index
        batch_size: Number of rows to fetch
        split: Dataset split name
    
    Returns:
        List of row dictionaries
    """
    data = fetch_rows(dataset_name, offset=start_idx, length=batch_size, split=split)
    if data:
        return [row.get('row', {}) for row in data.get('rows', [])]
    return []

def fetch_all_data(dataset_name, total_rows=None, batch_size=100, split="train"):
    """
    Fetch all data from dataset in batches.
    
    Args:
        dataset_name: Name of the dataset
        total_rows: Total number of rows (if None, will fetch until empty)
        batch_size: Number of rows per batch
        split: Dataset split name
    
    Returns:
        List of all row dictionaries
    """
    all_data = []
    offset = 0
    
    while True:
        batch = fetch_dataset_batch(dataset_name, offset, batch_size, split)
        if not batch:
            break
        
        all_data.extend(batch)
        offset += len(batch)
        
        print(f"Fetched {len(all_data)} rows so far...")
        
        if total_rows and len(all_data) >= total_rows:
            all_data = all_data[:total_rows]
            break
    
    return all_data

# Example: Fetch data in batches
print("=" * 70)
print("Method 2: Fetch Data in Batches")
print("=" * 70)

# Fetch first 50 rows in batches of 20
batches = []
for i in range(0, 50, 20):
    batch = fetch_dataset_batch("rotten_tomatoes", start_idx=i, batch_size=20, split="train")
    if batch:
        batches.extend(batch)
        print(f"Fetched batch starting at index {i}: {len(batch)} rows")

print(f"\nTotal rows fetched: {len(batches)}")
if batches:
    print(f"Sample row keys: {list(batches[0].keys())}")

In [ ]:
# ============================================================================
# Method 3: Get Dataset Size First, Then Fetch
# ============================================================================

def get_dataset_size(dataset_name, config=None, split=None):
    """
    Get the size (number of rows) of a dataset.
    
    Args:
        dataset_name: Name of the dataset
        config: Optional config name
        split: Optional split name
    
    Returns:
        Number of rows, or None if error
    """
    url = f"{BASE}/size"
    params = {"dataset": dataset_name}
    
    if config:
        params["config"] = config
    if split:
        params["split"] = split
    
    r = requests.get(url, params=params)
    if r.status_code == 200:
        data = r.json()
        if split:
            return data.get("splits", {}).get(split, {}).get("num_rows", 0)
        else:
            # Return total across all splits
            splits = data.get("splits", {})
            return sum(split_info.get("num_rows", 0) for split_info in splits.values())
    else:
        print(f"Error {r.status_code}: {r.text}")
        return None

# Example: Get size and fetch data
print("=" * 70)
print("Method 3: Get Size Then Fetch")
print("=" * 70)

dataset_name = "rotten_tomatoes"
size = get_dataset_size(dataset_name, split="train")
print(f"Dataset: {dataset_name}")
print(f"Train split size: {size} rows")

if size:
    # Fetch first 10 rows
    data = fetch_rows(dataset_name, offset=0, length=10, split="train")
    if data:
        print(f"\nFetched {len(data.get('rows', []))} rows")
        print(f"Total available: {size} rows")

In [ ]:
# ============================================================================
# Method 4: Download Parquet Files (For Large Datasets)
# ============================================================================

import pandas as pd
from io import BytesIO

def download_parquet_data(dataset_name, split="train", max_rows=None):
    """
    Download and load data from parquet files.
    
    Args:
        dataset_name: Name of the dataset
        split: Dataset split name
        max_rows: Maximum number of rows to load (None for all)
    
    Returns:
        pandas DataFrame with the data
    """
    # Get parquet URLs
    parquet_info = get_parquet_urls(dataset_name)
    
    if not parquet_info:
        return None
    
    # Find the parquet file for the requested split
    parquet_files = parquet_info.get("parquet", [])
    target_file = None
    
    for file_info in parquet_files:
        if file_info.get("split") == split:
            target_file = file_info
            break
    
    if not target_file:
        print(f"Split '{split}' not found in parquet files")
        return None
    
    parquet_url = target_file["url"]
    print(f"Downloading parquet file: {parquet_url}")
    print(f"Size: {target_file['size_bytes'] / 1e6:.1f} MB")
    
    # Download and read parquet
    response = requests.get(parquet_url)
    if response.status_code == 200:
        parquet_data = BytesIO(response.content)
        df = pd.read_parquet(parquet_data)
        
        if max_rows:
            df = df.head(max_rows)
        
        return df
    else:
        print(f"Error downloading: {response.status_code}")
        return None

# Example: Download parquet data
print("=" * 70)
print("Method 4: Download Parquet Files")
print("=" * 70)

# Note: This downloads the entire file, use with caution for large datasets
# For demo, we'll just show the URL structure
parquet_info = get_parquet_urls("rotten_tomatoes")
if parquet_info:
    print("Available parquet files:")
    for file_info in parquet_info.get("parquet", []):
        print(f"  Split: {file_info.get('split')}")
        print(f"  URL: {file_info.get('url')}")
        print(f"  Size: {file_info.get('size_bytes', 0) / 1e6:.1f} MB")
        print()

In [ ]:
# ============================================================================
# Method 5: Complete Example - Fetch and Process Data
# ============================================================================

def fetch_and_process_dataset(dataset_name, num_rows=100, split="train"):
    """
    Complete example: Fetch data and process it.
    
    Args:
        dataset_name: Name of the dataset
        num_rows: Number of rows to fetch
        split: Dataset split name
    
    Returns:
        List of processed row dictionaries
    """
    print(f"Fetching {num_rows} rows from {dataset_name} ({split} split)...")
    
    # Fetch data in batches
    all_rows = []
    batch_size = 50
    
    for offset in range(0, num_rows, batch_size):
        batch_length = min(batch_size, num_rows - offset)
        data = fetch_rows(dataset_name, offset=offset, length=batch_length, split=split)
        
        if data:
            rows = data.get("rows", [])
            for row_info in rows:
                row_data = row_info.get("row", {})
                all_rows.append(row_data)
        
        print(f"  Progress: {len(all_rows)}/{num_rows} rows")
    
    return all_rows

# Example: Complete workflow
print("=" * 70)
print("Method 5: Complete Example")
print("=" * 70)

dataset_name = "rotten_tomatoes"
rows = fetch_and_process_dataset(dataset_name, num_rows=20, split="train")

print(f"\nFetched {len(rows)} rows")
if rows:
    print(f"\nSample row:")
    print(f"  Keys: {list(rows[0].keys())}")
    print(f"  First row data: {rows[0]}")

In [10]:
# ============================================================================
# Method 6: Streaming Iterator (Memory Efficient)
# ============================================================================

def stream_dataset(dataset_name, split="train", batch_size=100):
    """
    Stream dataset rows one by one (memory efficient).
    
    Args:
        dataset_name: Name of the dataset
        split: Dataset split name
        batch_size: Number of rows to fetch per request
    
    Yields:
        Row dictionaries
    """
    offset = 0
    
    while True:
        data = fetch_rows(dataset_name, offset=offset, length=batch_size, split=split)
        
        if not data:
            break
        
        rows = data.get("rows", [])
        if not rows:
            break
        
        for row_info in rows:
            yield row_info.get("row", {})
        
        offset += len(rows)
        
        # Stop if we got fewer rows than requested (end of dataset)
        if len(rows) < batch_size:
            break

# Example: Stream dataset
print("=" * 70)
print("Method 6: Streaming Iterator (Memory Efficient)")
print("=" * 70)

count = 0
for row in stream_dataset("rotten_tomatoes", split="train", batch_size=10):
    count += 1
    if count <= 3:
        print(f"Row {count}: {list(row.keys())}")
    if count >= 10:
        break

print(f"\nStreamed {count} rows")

Method 6: Streaming Iterator (Memory Efficient)


NameError: name 'fetch_rows' is not defined

## Summary: Methods to Fetch Data from HuggingFace Datasets-Server

### API Endpoints:

1. **`/rows`** - Fetch specific rows
   - Parameters: `dataset`, `offset`, `length`, `config`, `split`
   - Best for: Fetching specific ranges of data

2. **`/first-rows`** - Get preview
   - Parameters: `dataset`, `config`, `split`, `rows`
   - Best for: Quick preview of dataset

3. **`/parquet`** - Get parquet file URLs
   - Parameters: `dataset`
   - Best for: Downloading entire dataset files

4. **`/size`** - Get dataset size
   - Parameters: `dataset`, `config`, `split`
   - Best for: Knowing dataset size before fetching

### When to Use Each Method:

- **`/rows`**: When you need specific rows or streaming
- **`/first-rows`**: Quick preview or exploration
- **`/parquet`**: Large datasets, full download
- **`/size`**: Planning data fetching strategy

In [21]:
import torch.nn as nn
from torch import Tensor
import torch
class SimpleModel(nn.Module):
   def __init__(self, vocab_size: int, embedding_dim: int):
      super().__init__()
      self.embedding = nn.Embedding(vocab_size, embedding_dim)

   def forward(self, x: Tensor):
      return self.embedding(x)
      

model = SimpleModel(vocab_size=100, embedding_dim=10)

x = torch.randint(0, 100, (1, 10, 10))
print(x)

print(model(x))

tensor([[[33, 65, 16, 84, 20, 79, 42, 76, 57, 64],
         [14, 70, 37, 84, 29, 43, 34, 72, 45, 83],
         [61, 15, 27, 91, 25, 72,  2, 11, 88, 62],
         [52, 40, 45, 60, 11, 60, 10, 25, 37,  8],
         [52, 94, 74, 60, 59, 87, 79, 84, 68, 56],
         [91,  4, 47, 98, 26, 56, 53, 10,  1, 58],
         [34, 50, 48, 95, 26, 43, 28, 67, 27, 96],
         [82, 35, 94, 58, 65, 97, 76, 66, 33, 26],
         [ 1, 69, 49, 34, 33, 23, 14, 12, 23,  2],
         [88, 14, 54, 13, 81, 21, 53, 54, 10, 70]]])
tensor([[[[ 3.7258e-01, -9.1141e-01, -6.2239e-01,  8.4786e-01, -1.1964e+00,
            1.5045e+00, -8.6630e-01,  9.3673e-01, -1.1734e+00,  5.2670e-01],
          [ 8.1614e-01,  5.3251e-01,  1.8865e+00, -1.6444e+00, -5.1596e-01,
            3.5914e-01, -7.0771e-01, -9.1873e-01, -5.8165e-01, -4.7662e-01],
          [-1.4621e+00, -5.9757e-01, -4.4163e-01, -1.1640e+00,  2.0024e-01,
            8.3701e-01,  1.8459e+00,  3.3611e-01, -2.8747e-02,  1.7438e+00],
          [ 1.3015e+00,  1.16